# Anthropic Extended Thinking with Claude

This notebook demonstrates Claude's extended thinking capabilities - a powerful feature that gives Claude enhanced reasoning for complex tasks.

## What is Extended Thinking?

Extended thinking allows Claude to "think" through complex problems step-by-step before providing a final answer. This is particularly useful for:

- **Mathematical problems** requiring multi-step calculations
- **Complex analysis** of documents or data
- **Architecture decisions** in software development
- **Strategic planning** and decision-making

### How It Works

When extended thinking is enabled, Claude:
1. Creates internal "thinking" content blocks
2. Works through the problem systematically
3. Incorporates insights from this reasoning
4. Delivers a more thoughtful final response

## Table of Contents

1. [Setup](#setup)
2. [Supported Models](#models)
3. [Basic Usage](#basic)
4. [Budget Tokens Parameter](#budget)
5. [Streaming Responses](#streaming)
6. [Tool Use with Extended Thinking](#tools)
7. [Interleaved Thinking (Beta)](#interleaved)
8. [Summarized vs Full Thinking](#summarized)
9. [Best Practices](#best-practices)
10. [Practical Examples](#examples)
11. [Summary](#summary)

<a id='setup'></a>
## 1. Setup

First, let's install the necessary packages and set up our API key.

We'll load the Anthropic API key, create the client, and verify initialization.
Expected output: a single line confirming the client is ready.

In [ ]:
import os
import getpass
import anthropic
from IPython.display import Markdown, display
import json
import time
from dotenv import load_dotenv

# Load environment variables from .env file (if it exists)
load_dotenv()

# Set up your API key
api_key = os.getenv('ANTHROPIC_API_KEY')
if not api_key:
    api_key = getpass.getpass("Enter your Anthropic API key: ")

client = anthropic.Anthropic(api_key=api_key)
print("Anthropic client initialized (ready for API calls)")

<a id='models'></a>
## 2. Supported Models

Extended thinking is supported in these Claude models:

| Model | Model ID | Notes |
|-------|----------|-------|
| **Claude Opus 4.5** | `claude-opus-4-5-20251101` | Latest flagship model |
| **Claude Sonnet 4.5** | `claude-sonnet-4-5-20250929` | Enhanced reasoning |
| **Claude Haiku 4.5** | `claude-haiku-4-5-20251001` | Fast model with thinking |
| **Claude Opus 4.1** | `claude-opus-4-1-20250805` | Advanced reasoning |
| **Claude Opus 4** | `claude-opus-4-20250514` | Powerful reasoning |
| **Claude Sonnet 4** | `claude-sonnet-4-20250514` | Balanced performance |

**Important**: Claude 4+ models return **summarized thinking** (you're charged for full internal tokens), while Claude 3.7 returns full thinking output (deprecated).

This table shows which Claude models support extended thinking and the IDs to use.
Pick one model ID and reuse it consistently while you learn.

<a id='basic'></a>
## 3. Basic Usage

Let's start with a simple example to see extended thinking in action.

This first example shows the structure of a response with thinking and final text.
You will see a short thinking summary and then the final answer.

In [ ]:
def basic_thinking_example():
    """A simple example demonstrating extended thinking"""
    
    print("Basic extended thinking example")
    print("-" * 60)
    
    response = client.messages.create(
        model="claude-sonnet-4-5-20250929",
        max_tokens=10000,
        thinking={
            "type": "enabled",
            "budget_tokens": 5000  # How many tokens Claude can use for thinking
        },
        messages=[{
            "role": "user",
            "content": "What is 27 * 453? Show me how you calculate this step by step."
        }]
    )
    
    # Process the response
    for block in response.content:
        if block.type == "thinking":
            print("Claude's Thinking Process (Summary):")
            print("-" * 50)
            print(block.thinking)
            print("-" * 50)
            print()
        elif block.type == "text":
            print("Final Answer:")
            print(block.text)

# Run the example
basic_thinking_example()

<a id='budget'></a>
## 4. Budget Tokens Parameter

The `budget_tokens` parameter determines how many tokens Claude can use for reasoning.

### Key Rules

- **Minimum**: 1,024 tokens
- **`max_tokens` must exceed `budget_tokens`** to allow room for the final response
- Claude may not use the entire budget - it only uses what's needed

### Budget Guidelines

| Budget Range | Use Case |
|--------------|----------|
| 1,024 - 5,000 | Basic reasoning tasks, simple calculations |
| 5,000 - 15,000 | Standard complex problems, code generation |
| 15,000 - 32,000 | Deep analysis, research, architecture planning |
| 32,000+ | Extensive multi-faceted problems (use batch API) |

We'll run the same prompt with different `budget_tokens` values and compare:
- runtime
- output length
- token usage

In [ ]:
def budget_comparison():
    """Compare different thinking budgets"""
    
    problem = """Analyze this business scenario:
    A coffee shop has 3 locations. Location A makes $2,500/day, 
    Location B makes $1,800/day, and Location C makes $3,200/day.
    Operating costs are 65% of revenue. They want to open a 4th location.
    What factors should they consider and what's the minimum daily revenue 
    the new location needs to be profitable?"""
    
    budgets = [1024, 5000, 15000]
    
    print("Thinking Budget Comparison (compare time, length, tokens)")
    print("=" * 60)
    
    for budget in budgets:
        print(f"\nBudget: {budget:,} tokens")
        print("-" * 40)
        
        start_time = time.time()
        
        # max_tokens must be greater than budget_tokens
        max_tokens = budget + 4000
        
        response = client.messages.create(
            model="claude-sonnet-4-5-20250929",
            max_tokens=max_tokens,
            thinking={"type": "enabled", "budget_tokens": budget},
            messages=[{"role": "user", "content": problem}]
        )
        
        elapsed_time = time.time() - start_time
        
        # Get response length and usage
        response_text = ""
        for block in response.content:
            if block.type == "text":
                response_text = block.text
        
        print(f"Time: {elapsed_time:.2f} seconds")
        print(f"Response length: {len(response_text)} characters")
        print(f"Tokens used - Input: {response.usage.input_tokens}, Output: {response.usage.output_tokens}")
        print(f"Response preview: {response_text[:200]}...")

budget_comparison()

<a id='streaming'></a>
## 5. Streaming Responses

For better user experience, especially with longer thinking times, you can stream responses.

**Important**: Streaming is **required** when `max_tokens` exceeds 21,333.

Streaming shows progress as thinking happens. Watch for dots during thinking and
then a full final response when the model switches to text output.

In [ ]:
def stream_thinking_example():
    """Demonstrate streaming with extended thinking"""
    
    print("Streaming Extended Thinking Example (dots = thinking)")
    print("=" * 60)
    
    with client.messages.stream(
        model="claude-sonnet-4-5-20250929",
        max_tokens=12000,
        thinking={"type": "enabled", "budget_tokens": 10000},
        messages=[{
            "role": "user",
            "content": """Design a simple REST API for a todo list application. 
            Include endpoints for CRUD operations and consider:
            - Authentication
            - Error handling
            - Data validation
            - Response formats"""
        }],
    ) as stream:
        current_block_type = None
        
        for event in stream:
            if event.type == "content_block_start":
                current_block_type = event.content_block.type
                if current_block_type == "thinking":
                    print("\nClaude is thinking...", end="", flush=True)
                elif current_block_type == "text":
                    print("\n\nFinal Response:\n", end="", flush=True)
            
            elif event.type == "content_block_delta":
                if event.delta.type == "thinking_delta":
                    # Show progress dots for thinking
                    print(".", end="", flush=True)
                elif event.delta.type == "text_delta":
                    print(event.delta.text, end="", flush=True)
            
            elif event.type == "content_block_stop":
                if current_block_type == "thinking":
                    print(" Done thinking!")

stream_thinking_example()

<a id='tools'></a>
## 6. Tool Use with Extended Thinking

Extended thinking can be combined with tool use for more powerful applications.

### Important Constraints

- Extended thinking only supports `tool_choice: "auto"` or `tool_choice: "none"`
- **Forced tool use is NOT supported** with extended thinking
- When continuing conversations, thinking blocks must be preserved unchanged

This example includes a tool call. Look for the tool input and then the final plan.

In [ ]:
def thinking_with_tools_example():
    """Demonstrate extended thinking with tool use"""
    
    # Define a simple calculator tool
    tools = [{
        "name": "calculator",
        "description": "Perform mathematical calculations",
        "input_schema": {
            "type": "object",
            "properties": {
                "expression": {
                    "type": "string",
                    "description": "Mathematical expression to evaluate"
                }
            },
            "required": ["expression"]
        }
    }]
    
    response = client.messages.create(
        model="claude-sonnet-4-5-20250929",
        max_tokens=20000,
        thinking={
            "type": "enabled",
            "budget_tokens": 8000
        },
        tools=tools,
        tool_choice={"type": "auto"},  # Only auto or none supported
        messages=[{
            "role": "user",
            "content": """I'm planning a party for 25 people. Each person will eat:
            - 3 slices of pizza (8 slices per pizza)
            - 2 sodas ($1.50 each)
            - 1 dessert ($3.00 each)
            
            Pizzas cost $12 each. Calculate the total cost and quantities needed."""
        }]
    )
    
    print("Party Planning with Extended Thinking (tool calls + final plan)")
    print("=" * 60)
    
    for block in response.content:
        if block.type == "thinking":
            print("\nPlanning Process:")
            print(block.thinking[:500] + "...\n")
        elif block.type == "tool_use":
            print(f"\nUsing tool: {block.name}")
            print(f"   Input: {block.input}")
        elif block.type == "text":
            print("\nFinal Plan:")
            display(Markdown(block.text))

thinking_with_tools_example()

<a id='interleaved'></a>
## 7. Interleaved Thinking (Beta)

A powerful new feature for **Claude 4 models only** that enables reasoning between tool calls.

### How to Enable

Use the beta header: `anthropic-beta: interleaved-thinking-2025-05-14`

### Benefits

- Claude can think about tool results before deciding next steps
- Enables more sophisticated multi-step workflows
- Better tool orchestration for complex tasks

Interleaved thinking lets Claude reason between multiple tool calls.
This cell is safe to read without running; uncomment only if you have access.

In [ ]:
def interleaved_thinking_example():
    """Example of interleaved thinking with multiple tool calls"""
    
    # Client with beta header for interleaved thinking
    client_with_beta = anthropic.Anthropic(
        api_key=api_key,
        default_headers={
            "anthropic-beta": "interleaved-thinking-2025-05-14"
        }
    )
    
    tools = [
        {
            "name": "search_database",
            "description": "Search for information in a database",
            "input_schema": {
                "type": "object",
                "properties": {
                    "query": {"type": "string"},
                    "filters": {"type": "object"}
                },
                "required": ["query"]
            }
        },
        {
            "name": "analyze_data",
            "description": "Analyze data and return insights",
            "input_schema": {
                "type": "object",
                "properties": {
                    "data": {"type": "array"},
                    "analysis_type": {"type": "string"}
                },
                "required": ["data", "analysis_type"]
            }
        }
    ]
    
    # max_tokens must be greater than budget_tokens
    budget = 16384
    
    response = client_with_beta.messages.create(
        model="claude-sonnet-4-5-20250929",
        max_tokens=budget + 4000,
        tools=tools,
        messages=[{
            "role": "user",
            "content": "Find all customers who made purchases last month and analyze their buying patterns."
        }],
        thinking={
            "type": "enabled",
            "budget_tokens": budget  # Larger budget for complex multi-step reasoning
        }
    )
    
    print("Interleaved Thinking Example (tool reasoning between calls)")
    print("=" * 60)
    print("\nWith interleaved thinking, Claude can:")
    print("1. Think about what data to search for")
    print("2. Call search_database tool")
    print("3. Think about the results")
    print("4. Decide to call analyze_data tool")
    print("5. Think about the analysis")
    print("6. Provide final insights")
    
    for block in response.content:
        if block.type == "thinking":
            print(f"\n[THINKING]: {block.thinking[:200]}...")
        elif block.type == "tool_use":
            print(f"\n[TOOL CALL]: {block.name}")
        elif block.type == "text":
            print(f"\n[RESPONSE]: {block.text[:200]}...")
    
    return response

# Note: Uncomment to run (requires Claude 4 model access)
# interleaved_thinking_example()

<a id='summarized'></a>
## 8. Summarized vs Full Thinking

### Claude 4+ Models (Current)

- Returns **summarized thinking** in the `thinking` block
- You are **billed for full internal thinking tokens**, not just the summary
- Summary provides insight into reasoning without exposing full internal process

### Claude 3.7 (Deprecated)

- Returned **full thinking output**
- What you saw is what you were charged for

### Key Implications

- Thinking blocks from previous turns are **stripped** and don't count toward context window
- Each thinking block includes a **cryptographic signature** for verification
- Don't parse or modify the signature field

This section clarifies billing and what you see in the response. Keep this in mind
when you estimate cost or compare output quality.

In [ ]:
def analyze_thinking_blocks():
    """Demonstrate the structure of thinking blocks"""
    
    response = client.messages.create(
        model="claude-sonnet-4-5-20250929",
        max_tokens=15000,
        thinking={
            "type": "enabled",
            "budget_tokens": 8000
        },
        messages=[{
            "role": "user",
            "content": """I have a list of numbers: [15, 23, 8, 42, 16, 4, 30, 12].
            
            Please:
            1. Find the median
            2. Calculate the mean
            3. Identify any outliers using the IQR method"""
        }]
    )
    
    print("Response Structure Analysis (block types and sizes)")
    print("=" * 60)
    
    for i, block in enumerate(response.content):
        print(f"\nBlock {i + 1}:")
        print(f"  Type: {block.type}")
        
        if block.type == "thinking":
            print(f"  Thinking Summary Length: {len(block.thinking)} characters")
            print(f"  Has Signature: {'Yes' if hasattr(block, 'signature') else 'No'}")
            print("\n  Thinking Content (first 500 chars):")
            print(f"  {block.thinking[:500]}...")
            print("\n  Note: This is a summary. Full thinking tokens were used internally.")
        elif block.type == "text":
            print(f"  Text Length: {len(block.text)} characters")
            print("\n  Final Response:")
            display(Markdown(block.text))

analyze_thinking_blocks()

<a id='best-practices'></a>
## 9. Best Practices

### Start with Minimal Budget

Begin with 1,024 tokens and increase only as needed for the task.

### Don't Say "Think Step by Step"

Extended thinking handles this automatically - explicit instructions can be counterproductive.

### Use High-Level Instructions

Claude performs better with general guidance rather than step-by-step directives.

### Verify Work with Test Cases

Ask Claude to check its reasoning for better consistency.

### Incompatible Settings

Extended thinking is **NOT compatible** with:
- `temperature` parameter
- `top_k` parameter
- Forced tool use (`tool_choice: {type: "tool", name: "..."}`)

A practical prompting example. Read the prompt structure first, then run the cell
and examine the model's response format.

In [ ]:
def prompting_best_practices():
    """Demonstrate effective prompting strategies"""
    
    # Good prompt - clear, specific, structured
    good_prompt = """Analyze the following investment options and recommend the best choice:

Option A: Stock Portfolio
- Expected annual return: 8%
- Risk level: High
- Minimum investment: $10,000
- Liquidity: High (can sell anytime)

Option B: Real Estate
- Expected annual return: 6%
- Risk level: Medium
- Minimum investment: $50,000
- Liquidity: Low (takes months to sell)

Option C: Bonds
- Expected annual return: 4%
- Risk level: Low
- Minimum investment: $5,000
- Liquidity: Medium

Investor Profile:
- Age: 35
- Investment horizon: 15 years
- Risk tolerance: Medium
- Available capital: $75,000
- Goal: Retirement savings

Please provide:
1. Analysis of each option
2. Recommended allocation
3. Justification for your recommendation"""
    
    # max_tokens must be greater than budget_tokens
    budget = 10000
    
    response = client.messages.create(
        model="claude-sonnet-4-5-20250929",
        max_tokens=budget + 4000,
        thinking={"type": "enabled", "budget_tokens": budget},
        messages=[{"role": "user", "content": good_prompt}]
    )
    
    print("Best Practices Example: Structured Investment Analysis")
    print("=" * 60)
    
    for block in response.content:
        if block.type == "text":
            display(Markdown(block.text))

prompting_best_practices()

In [ ]:
def cost_calculator():
    """Calculate costs for extended thinking usage"""
    
    # Current pricing (prices per million tokens)
    pricing = {
        "claude-sonnet-4-5": {"input": 3, "output": 15},
        "claude-haiku-4-5": {"input": 1, "output": 5},
        "claude-opus-4-1": {"input": 15, "output": 75},
        "claude-sonnet-4": {"input": 3, "output": 15},
    }
    
    print("Extended Thinking Cost Calculator (approximate costs)")
    print("=" * 60)
    
    # Example scenario
    scenarios = [
        {"name": "Simple Analysis", "input": 500, "thinking": 5000, "output": 1000},
        {"name": "Complex Problem", "input": 2000, "thinking": 20000, "output": 3000},
        {"name": "Deep Research", "input": 5000, "thinking": 50000, "output": 8000}
    ]
    
    # Show example with Sonnet 4.5
    model = "claude-sonnet-4-5"
    prices = pricing[model]
    
    print(f"\nCost Analysis for {model}")
    print("-" * 40)
    
    for scenario in scenarios:
        # Thinking tokens are billed as output tokens
        input_cost = (scenario["input"] / 1_000_000) * prices["input"]
        thinking_cost = (scenario["thinking"] / 1_000_000) * prices["output"]
        output_cost = (scenario["output"] / 1_000_000) * prices["output"]
        total_cost = input_cost + thinking_cost + output_cost
        
        print(f"\n  {scenario['name']}:")
        print(f"    Input tokens: {scenario['input']:,}")
        print(f"    Thinking tokens: {scenario['thinking']:,} (billed as output)")
        print(f"    Output tokens: {scenario['output']:,}")
        print(f"    Total cost: ${total_cost:.4f}")
    
    print("\n\nImportant Pricing Notes:")
    print("-" * 40)
    print("- Thinking tokens are charged at OUTPUT token rates")
    print("- Claude 4+ models: Charged for full internal thinking, not summary")
    print("- Thinking blocks from previous turns don't count toward context window")
    print("- Use prompt caching to reduce costs on repeated patterns")

cost_calculator()

<a id='examples'></a>
## 10. Practical Examples

Three longer examples (document analysis, architecture planning, and code review).
Use them as reference patterns for real-world tasks.

In [ ]:
def document_analysis_example():
    """Analyze a complex document with extended thinking"""
    
    # Simulated legal document excerpt
    document = """PURCHASE AGREEMENT - EXECUTIVE SUMMARY
    
    This Agreement is entered into as of January 15, 2024, between TechCorp Inc. 
    ("Buyer") and DataSystems LLC ("Seller").
    
    TERMS:
    1. Purchase Price: $45,000,000 (Forty-five million dollars)
    2. Payment Structure:
       - Initial Payment: $20,000,000 upon closing
       - Deferred Payment: $15,000,000 payable over 3 years
       - Performance Earnout: Up to $10,000,000 based on revenue targets
    
    3. Conditions Precedent:
       - Regulatory approval from FTC
       - No material adverse change in Seller's business
       - Retention of key employees (minimum 80% for 12 months)
    
    4. Representations and Warranties:
       - Seller warrants all intellectual property is free of encumbrances
       - Financial statements are accurate per GAAP
       - No pending litigation exceeding $500,000
    
    5. Termination Clauses:
       - Either party may terminate if closing doesn't occur by March 31, 2024
       - Buyer may terminate if due diligence reveals material issues
       - Break-up fee: $2,000,000 if Buyer terminates without cause
    """
    
    # max_tokens must be greater than budget_tokens
    budget = 10000
    
    response = client.messages.create(
        model="claude-sonnet-4-5-20250929",
        max_tokens=budget + 4000,
        thinking={"type": "enabled", "budget_tokens": budget},
        messages=[{
            "role": "user",
            "content": f"""Analyze this purchase agreement and identify:
            
            1. Key risks for the buyer
            2. Key risks for the seller
            3. Potential deal breakers
            4. Areas that need clarification
            5. Recommendations for both parties
            
            Document:
            {document}"""
        }]
    )
    
    print("Complex Document Analysis with Extended Thinking")
    print("=" * 60)
    
    for block in response.content:
        if block.type == "thinking":
            print("\nAnalysis Process (Summary):")
            print(block.thinking[:800] + "...\n")
        elif block.type == "text":
            print("Detailed Analysis:")
            display(Markdown(block.text))

document_analysis_example()

In [ ]:
def architecture_planning_example():
    """Use extended thinking for software architecture decisions"""
    
    requirements = """Design a microservices architecture for an e-commerce platform with:
    
    Functional Requirements:
    - User authentication and profiles
    - Product catalog with search
    - Shopping cart and checkout
    - Order management and tracking
    - Payment processing
    
    Non-Functional Requirements:
    - Handle 100,000 concurrent users
    - 99.9% uptime
    - Response time < 200ms for catalog
    - Scalable to 10x current load
    
    Tech Stack Preferences:
    - Cloud-native (AWS/GCP/Azure)
    - Container-based deployment
    - Modern languages (Python/Go/Node.js)
    """
    
    # max_tokens must be greater than budget_tokens
    budget = 15000
    
    response = client.messages.create(
        model="claude-sonnet-4-5-20250929",
        max_tokens=budget + 5000,
        thinking={"type": "enabled", "budget_tokens": budget},
        messages=[{
            "role": "user",
            "content": f"""Create a detailed microservices architecture plan.
            
            Include:
            1. Service breakdown and responsibilities
            2. Communication patterns (sync/async)
            3. Data storage strategy
            4. Security considerations
            5. Deployment architecture
            6. Scaling strategy
            
            Requirements:
            {requirements}"""
        }]
    )
    
    print("Microservices Architecture Planning (long-form response)")
    print("=" * 60)
    
    for block in response.content:
        if block.type == "text":
            display(Markdown(block.text))

architecture_planning_example()

In [ ]:
def code_review_example():
    """Code review with extended thinking"""
    
    code = '''
def process_transactions(transactions, user_id):
    results = []
    total = 0
    
    for t in transactions:
        if t["user"] == user_id:
            amount = t["amount"]
            if t["type"] == "credit":
                total += amount
            else:
                total -= amount
            results.append({"id": t["id"], "running_total": total})
    
    return results, total

def get_user_balance(user_id):
    conn = database.connect()
    cursor = conn.cursor()
    cursor.execute(f"SELECT * FROM transactions WHERE user_id = {user_id}")
    rows = cursor.fetchall()
    transactions = [{"id": r[0], "user": r[1], "amount": r[2], "type": r[3]} for r in rows]
    return process_transactions(transactions, user_id)
'''
    
    # max_tokens must be greater than budget_tokens
    budget = 10000
    
    response = client.messages.create(
        model="claude-sonnet-4-5-20250929",
        max_tokens=budget + 4000,
        thinking={"type": "enabled", "budget_tokens": budget},
        messages=[{
            "role": "user",
            "content": f"""Review this Python code for:
            1. Security vulnerabilities
            2. Performance issues
            3. Best practice violations
            4. Error handling gaps
            
            Provide specific fixes for each issue found.
            
            Code:
            ```python
            {code}
            ```"""
        }]
    )
    
    print("Code Review with Extended Thinking (issues + fixes)")
    print("=" * 60)
    
    for block in response.content:
        if block.type == "text":
            display(Markdown(block.text))

code_review_example()

<a id='summary'></a>
## 11. Summary

### What We've Learned

1. **Extended Thinking Basics**: How to enable and use Claude's reasoning capabilities
2. **Model Support**: All Claude 4+ models support extended thinking
3. **Budget Tokens**: Start minimal (1,024) and increase based on task complexity
4. **Streaming**: Required for large max_tokens, improves UX
5. **Tool Use**: Only `tool_choice: "auto"` or `"none"` supported
6. **Interleaved Thinking**: Beta feature for reasoning between tool calls (Claude 4 only)
7. **Summarized Thinking**: Claude 4+ returns summaries, billed for full tokens

### Key Takeaways

**Use Extended Thinking for:**
- Complex multi-step problems requiring sequential reasoning
- Deep document analysis and structured evaluation
- Strategic planning with multiple constraints
- STEM problems and optimization tasks
- Quality-critical tasks where accuracy matters more than speed

**Avoid Extended Thinking for:**
- Simple queries or lookups
- Real-time chat applications
- Tasks where latency is critical
- High-volume, low-complexity requests

### Common Pitfalls to Avoid

- Don't toggle thinking mid-conversation (complete the assistant turn first)
- Don't use with forced tool use or modified temperature settings
- Don't manually edit or parse signature fields
- Don't over-specify instructions (let Claude's creativity work)
- Don't forget to pass thinking blocks back unchanged when using tools

### Resources

- [Extended Thinking Documentation](https://docs.anthropic.com/en/docs/build-with-claude/extended-thinking)
- [Extended Thinking Tips](https://docs.anthropic.com/en/docs/build-with-claude/prompt-engineering/extended-thinking-tips)
- [Anthropic API Reference](https://docs.anthropic.com/api/)